In [1]:
import os
import pandas as pd
import numpy as np
import json
import re
from modules import *
from tqdm import tqdm
from utils import query_gpt, parse_and_select, read_config, login_gis_portal
from utils import get_prompt_for_extracting_modules

%load_ext autoreload
%autoreload 2

2025-05-27 20:16:22.728305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Get templates to extract module lists

In [ ]:
ask_num = 10

api_key = ''
prompt_version = '2'

In [3]:
V4 = [
    ('PL', 'Poland'),
    ('SK', 'Slovakia'),
    ('HU', 'Hungary'),
    ('CZ', 'Czech republic'),
]
# target_var = 'work_accidents'
# target_var_desc = 'the number of accidents at work'

# target_var = 'GRDP'
# target_var_desc = 'a regional GDP'

target_var = 'ELP'
target_var_desc = 'the elderly population above 64'

# target_var = 'population'
# target_var_desc = 'the population'

# target_var = 'education_high'
# target_var_desc = 'the proportion of highly-educated individuals (at least university graduates)'

# target_var = 'labour_force_participation_rate'
# target_var_desc = 'the labour force participation rate' 

# target_var = 'life_expectancy'
# target_var_desc = 'the life expectancy at birth'

# target_var = 'LPR'
# target_var_desc = 'the labour force participation rate' 

# target_var = 'life_expectancy_deviation'
# target_var_desc = 'the deviation of life expectancy at birth from the average'

In [4]:
V4

[('PL', 'Poland'),
 ('SK', 'Slovakia'),
 ('HU', 'Hungary'),
 ('CZ', 'Czech republic')]

In [5]:
if not os.path.isdir(f"selected_modules/{target_var}"):
    os.mkdir(f"selected_modules/{target_var}")
    os.mkdir(f"selected_modules/{target_var}/main")

In [6]:
for target_ccode, country_name in V4:
    example_question = f'Which information is useful to infer {target_var_desc} of {country_name}?'
    target_question = f"Infer {target_var_desc} as a score within [0, 1] from given location's description. Answer the score only."
    fewshot_target_question = f"Infer {target_var_desc} from given location's description. Answer the numeric score only."

    print(target_var)
    print('example_question :', example_question)
    print('target_question :', target_question)
    print('fewshot_target_question :', fewshot_target_question)
    
    prompt = get_prompt_for_extracting_modules(example_question, version=prompt_version)
    prompts = [prompt for _ in range(ask_num)]
#     print(prompts[0])
#     assert(1==0)
    answers = query_gpt(prompts, api_key, temperature=0.5, max_completion_tokens=20000, model='o3-mini', max_try_num=10)
    
    # with open(f'selected_modules/{target_var}/main/{target_ccode}_{target_var}_o3mini_wo_neighbor_raw.json','w') as f:
    #     json.dump({'s_modules':answers},f,indent=4)
    
    for answer in answers:
        try:
            x =answer.split('The total list of selected modules: ')

            print(x[1].split('\n'))
        except:
            pass
    ans_dict ={}
    for i in answers:
        modules = list(dict.fromkeys(re.findall(r'^\d.*\n', i, re.MULTILINE)))
        # print(modules)
        for module in modules:

            temp = module[3:].replace('`',"").replace('\n','')
            if temp in ans_dict:
                ans_dict[temp] +=1
            else:
                ans_dict[temp] =1

    s_modules = np.array([*ans_dict.keys()])[np.array([*ans_dict.values()]) >= len(answers) // 4]

    print(s_modules)
    if 'get_address(Loc)' not in s_modules:
        s_modules = np.insert(s_modules, 0, 'get_address(Loc)')
    elif np.where(s_modules == 'get_address(Loc)')!=0 :
        address_index = np.where(s_modules == 'get_address(Loc)')[0][0]
        s_modules = np.delete(s_modules, address_index)
        s_modules = np.insert(s_modules, 0, 'get_address(Loc)')
        
    s_modules = [elem.replace('Func=','').replace('Class=','class=').replace('_target(Loc, class=','_target(Loc, target_name=').replace('class=','target_class=').replace("\"","'").replace("get_landcover_ratio(Loc, '","get_landcover_ratio(Loc, target_class='").replace("get_distance_to_nearest_target(Loc, '","get_distance_to_nearest_target(Loc, target_name='").replace('Loc=x', 'loc=x') for elem in s_modules]
    with open(f'selected_modules/{target_ccode}_{target_var}_o3mini.json','w') as f:
        json.dump({'s_modules':list(s_modules)},f,indent=4)

ELP
example_question : Which information is useful to infer the elderly population above 64 of Poland?
target_question : Infer the elderly population above 64 as a score within [0, 1] from given location's description. Answer the score only.
fewshot_target_question : Infer the elderly population above 64 from given location's description. Answer the numeric score only.


 20%|██        | 2/10 [00:46<03:07, 23.39s/it]


KeyboardInterrupt: 